In [1]:
using LinearAlgebra
using Plots

include("Cholesky.jl")
include("Algoritmo LU.jl")

LUPP (generic function with 1 method)

# Problema 9

En un trabajo relacionado con el estudio de la eficiencia de la utilización de la energía por las larvas de la polilla modesta (Pachysphinx modesta), L. Schroeder [Schrl] utilizó los siguientes datos para determinar una relación entre $W$, el peso de las larvas vivas en gramos, y $R$, el consumo de oxígeno de las larvas en mililitros/hora. Por razones biológicas, se supone que entre $W$ y $R$ existe una relación de la forma $R = b W^a$.



\begin{array}{|c|c|c|c|c|c|c|c|} 
W & R & W & R & W & R & W & R  \\ 
0.017 & 0.154 & 0.211 & 0.366 & 3.040 & 3.590 & 0.233 & 0.537 \\ 
0.087 & 0.296 & 0.999 & 0.771 & 4.290 & 3.600 & 0.783 & 1.470 \\ 
0.174 & 0.363 & 3.020 & 2.010 & 5.300 & 3.880 & 1.350 & 2.480 \\ 
1.110 & 0.531 & 4.280 & 3.280 & 0.020 & 0.180 & 1.690 & 1.440 \\ 
1.740 & 2.230 & 4.580 & 2.960 & 0.119 & 0.299 & 2.750 & 1.840 \\ 
4.090 & 3.580 & 4.680 & 5.100 & 0.210 & 0.428 & 4.830 & 4.660 \\ 
5.450 & 3.520 & 0.020 & 0.181 & 1.320 & 1.150 & 5.530 & 6.940 \\ 
5.960 & 2.400 & 0.085 & 0.260 & 3.340 & 2.830 &  &  \\ 
0.025 & 0.23 & 0.171 & 0.334 & 5.480 & 4.150 &  &  \\ 
0.111 & 0.257 & 1.290 & 0.870 & 0.025 & 0.234 &  & 
\end{array}


# a)
Encuentre el polinomio logarítmico lineal de mínimos cuadrados
\begin{align}
\ln R = \ln b + a \ln W
\end{align}
utilizando el método de rotaciones de Givens.

Buscamos que
    \begin{align}
        \ln R_i & = \ln b + a \ln W_i \\
        & = \begin{pmatrix}
         1 & \ln W_i
        \end{pmatrix} \begin{pmatrix}
        \ln b \\ a
        \end{pmatrix}
    \end{align}
    
Por lo tanto, la matriz de diseño será la matriz de $37 \times 2$
    \begin{align}
        A = \begin{pmatrix} 
            1 & \ln W_1 \\
            1 & \ln W_2 \\
            \vdots \\
            1 & \ln W_{37}
        \end{pmatrix}
    \end{align}
y buscamos resolver el problema 
    \begin{align}
        A \beta = \ln R,
    \end{align}
donde
    \begin{align}
        \beta = \begin{pmatrix}
            \ln b \\
            a
        \end{pmatrix}
     \end{align}
y
    \begin{align}
        \ln R = \begin{pmatrix}
            \ln R_1 \\
            \ln R_2 \\
            \vdots \\
            \ln R_{37}
           \end{pmatrix}
     \end{align}

In [1]:
# Vectores de datos
# Peso de las larvas
W = [0.017; 0.087; 0.174; 1.110; 1.740; 4.090; 5.450; 5.960; 0.025; 
     0.111; 0.211; 3.040; 0.233; 0.999; 4.290; 0.783; 3.020; 5.300;
     1.350; 4.280; 0.020; 1.690; 4.580; 0.119; 2.750; 4.680; 0.210;
     4.830; 0.020; 1.320; 5.530; 0.085; 3.340; 0.171; 5.480; 1.290;
     0.025]
# Consumo de oxígeno en mililitros/hora
R = [0.154; 0.366; 3.590; 0.537; 0.296; 0.771; 3.600; 1.470; 0.363; 2.010; 
    3.880; 2.480; 0.531; 3.280; 0.180; 1.440; 2.230; 2.960; 0.299; 1.840;
    3.580; 5.100; 0.428; 4.660; 3.520; 0.181; 1.150; 6.940; 2.400; 0.260;
    2.830;  0.23; 0.334; 4.150; 0.257; 0.870; 0.234]

m = length(W)

# Matriz de diseño
A = ones(m,2)
A[:,2] = log.(W)

#ln R
Rlog = log.(R)

#Ahora necesitamos calcular la factorización QR de A mediante rotaciones de Givens.

# Esta función toma un vector x de nxn y dos índices i,j \leq n
# Regresa una matriz de rotación G tal que y = Gx tiene su j-ésima 
# entrada es igual a 0
function Givens(i,j,x)
    n = length(x)
    G = Matrix(1.0*I,n,n) 
    G[i,i] = x[i]/(sqrt(x[i]^2+x[j]^2))
    G[j,j] = G[i,i]
    G[j,i] = -x[j]/(sqrt(x[i]^2+x[j]^2))
    G[i,j] = -G[j,i]
    return(G)
end

# Esta función calcula la factorización QR de una matriz A mediante rotaciones de Givens
function QRGivens(A)
    (m,n) = size(A) 
    Q = Matrix(1.0*I,m,m)
    B = A
    for j = 1:n-1
        for i = m:-1:j+1
            x = B[:,j]
            T = Givens(j,i,x)  ### Estamos parados en la j-ésima columna y queremos hacer cero la i-ésima entrada (ver comentario de fnción Givens)
            B = T*B
            Q = T*Q
        end
    end
    Q = Q'
    R = B
    return(Q,R)
end

(Q, RComplete) = QRGivens(A)


# Ya que tenemos la factorización QR, la usamos para resolver el sistema de ecuaciones
# R* beta = Q^T ln(R)



LoadError: UndefVarError: I not defined

Ya tenemos la factorización QR de A. Ahora tenemos que resolver el sistema 
    \begin{align}
        \hat{R} \beta = Q^T \ln R
    \end{align}
mediante sustitución hacia atrás

In [2]:
beta = SolBwd(RComplete[1:2,1:2],(Q'*Rlog)[1:2,1])

lnb = beta[1]
a = beta[2]
b = exp(lnb)

println(a)
println(lnb)
println(b)

LoadError: UndefVarError: RComplete not defined

De esta manera, 
    \begin{align}
        a & = 0.5072524246351425 \\
        \ln b & = 0.2335109470838186 \\
        b & = 1.2630266542738828
    \end{align}
Por lo tanto,
    \begin{align}
        R & = bW^a \\
        & = e^{0.2335109470838186} W^{0.5072524246351425}
    \end{align}

# b)

Calcule el error asociado a la aproxmación en la parte (a)

In [3]:
E = 0

for i = 1:37
    E = E + (R[i] - b*(W[i]^a) )^2
end

println(E)

LoadError: UndefVarError: a not defined

# c)

Modifique la ecuación logarítmica de mínimos cuadrados de la parte (a), agregando el término cuadrático $c (\ln W_i)^2$, y después determine el polinomio logarítmico de mínimos cuadrados usando de nuevo el método de rotaciones de Givens. Calcule el error asociado a la aproximación.

El polinomio logarítmico es de la forma
    \begin{align}
        \ln R = \ln b_2 + a_2 \ln W + c_2 (\ln W)^2.
    \end{align}
    
Ahora la matriz de diseño será de la forma
    \begin{align}
         A = \begin{pmatrix} 
            1 & \ln W_1 & (\ln W_1 )^2 \\
            1 & \ln W_2 & (\ln W_2 )^2 \\
            \vdots \\
            1 & \ln W_{37} & (\ln W_{37} )^2
        \end{pmatrix}
    \end{align}
y buscamos resolver el problema 
    \begin{align}
        A \gamma = \ln R,
    \end{align}
donde
    \begin{align}
        \gamma = \begin{pmatrix}
            \ln b_2 \\
            a_2 \\
            c_2
        \end{pmatrix}
     \end{align}
y
    \begin{align}
        \ln R = \begin{pmatrix}
            \ln R_1 \\
            \ln R_2 \\
            \vdots \\
            \ln R_{37}
           \end{pmatrix}
     \end{align}

In [5]:
# Matriz de diseño
A2 = ones(m,3)
A2[:,2] = log.(W)
A2[:,3] = (log.(W)).*(log.(W))

#ln R
Rlog = log.(R)

#Ahora necesitamos calcular la factorización QR de A mediante rotaciones de Givens.

# Esta función toma un vector x de nxn y dos índices i,j \leq n
# Regresa una matriz de rotación G tal que y = Gx tiene su j-ésima 
# entrada es igual a 0
function Givens(i,j,x)
    n = length(x)
    G = Matrix(1.0*I,n,n) 
    G[i,i] = x[i]/(sqrt(x[i]^2+x[j]^2))
    G[j,j] = G[i,i]
    G[j,i] = -x[j]/(sqrt(x[i]^2+x[j]^2))
    G[i,j] = -G[j,i]
    return(G)
end

# Esta función calcula la factorización QR de una matriz A mediante rotaciones de Givens
function QRGivens(A)
    (m,n) = size(A) 
    Q = Matrix(1.0*I,m,m)
    B = A
    for j = 1:n-1
        for i = m:-1:j+1
            x = B[:,j]
            T = Givens(j,i,x)  ### Estamos parados en la j-ésima columna y queremos hacer cero la i-ésima entrada (ver comentario de fnción Givens)
            B = T*B
            Q = T*Q
        end
    end
    Q = Q'
    R = B
    return(Q,R)
end

(Q2, R2Complete) = QRGivens(A2)

([0.16439898730535726 -0.31678855141882806 … -0.16537264602577986 0.7048342842273488; 0.16439898730535726 -0.17799578662310012 … 0.8263967341365583 -0.12568710444616113; … ; 0.16439898730535735 0.051228451694382705 … 0.41409061453456675 0.0034913084568378105; 0.16439898730535726 -0.28400400232694906 … -0.31758436302016674 -0.6978516673136732], [6.082762530298219 -2.116715711899544 23.486277588634294; -6.519923962158026e-17 11.763543848999129 -21.959329885917384; … ; -9.301402406748514e-17 -1.7259041855875274e-16 -4.560404511318388; 1.4146550772786476e-17 -1.6525082561288963e-17 1.82839511089448])

In [6]:
beta2 = SolBwd(R2Complete[1:3,1:3],(Q2'*Rlog)[1:3,1])
lnb2 = beta2[1]
a2 = beta2[2]
c2 = beta2[3]

b2 = exp(lnb2)

println(a2)
println(lnb2)
println(b2)
println(c2)

-0.6312731855589997
1.2406940191527989
3.4580125591043056
-0.363462897311612


De esta manera,
    \begin{align}
        a_2 & = -0.6312731855589997 \\
        b_2 & = 3.4580125591043056 \\
        c_2 & = -0.363462897311612
    \end{align}
y el modelo es 
    \begin{align}
        R = b W^a \exp ( c (\ln W)^2)
    \end{align}

In [23]:
Equad = 0
for i = 1:37
   Equad = Equad + ( R[i] - b2*(W[i]^a2)*exp(c2*(log(W[i])^2)))^2
end
println(Equad)

163.1474067643412


# d)

Agregue ahora un término cúbico a la ecuación logarítmica y determine el polinomio logarítmico de mínimos cuadrados con rotaciones de Givens. Calcule también el error y concluya.

El polinomio logarítmico ahora es de la forma
    \begin{align}
        \ln R = \ln b_3 + a_3 \ln W + c_3 (\ln W)^2 + d_3 (\ln W)^3.
    \end{align}
    
Ahora la matriz de diseño será de la forma
    \begin{align}
         A = \begin{pmatrix} 
            1 & \ln W_1 & (\ln W_1 )^2 & (\ln W_1)^3\\
            1 & \ln W_2 & (\ln W_2 )^2 & (\ln W_2)^3 \\
            \vdots & \vdots & \vdots & \vdots \\
            1 & \ln W_{37} & (\ln W_{37} )^2 & (\ln W_{37})^3
        \end{pmatrix}
    \end{align}
y buscamos resolver el problema 
    \begin{align}
        A \delta = \ln R,
    \end{align}
donde
    \begin{align}
        \delta = \begin{pmatrix}
            \ln b_3 \\
            a_3 \\
            c_3 \\
            d_3
        \end{pmatrix}
     \end{align}
y
    \begin{align}
        \ln R = \begin{pmatrix}
            \ln R_1 \\
            \ln R_2 \\
            \vdots \\
            \ln R_{37}
           \end{pmatrix}
     \end{align}

In [24]:
# Matriz de diseño
A3 = ones(m,4)
A3[:,2] = log.(W)
A3[:,3] = (log.(W)).*(log.(W))
A3[:,4] = (log.(W)).^3
#ln R
Rlog = log.(R)

#Ahora necesitamos calcular la factorización QR de A mediante rotaciones de Givens.

# Esta función toma un vector x de nxn y dos índices i,j \leq n
# Regresa una matriz de rotación G tal que y = Gx tiene su j-ésima 
# entrada es igual a 0
function Givens(i,j,x)
    n = length(x)
    G = Matrix(1.0*I,n,n) 
    G[i,i] = x[i]/(sqrt(x[i]^2+x[j]^2))
    G[j,j] = G[i,i]
    G[j,i] = -x[j]/(sqrt(x[i]^2+x[j]^2))
    G[i,j] = -G[j,i]
    return(G)
end

# Esta función calcula la factorización QR de una matriz A mediante rotaciones de Givens
function QRGivens(A)
    (m,n) = size(A) 
    Q = Matrix(1.0*I,m,m)
    B = A
    for j = 1:n-1
        for i = m:-1:j+1
            x = B[:,j]
            T = Givens(j,i,x)  ### Estamos parados en la j-ésima columna y queremos hacer cero la i-ésima entrada (ver comentario de fnción Givens)
            B = T*B
            Q = T*Q
        end
    end
    Q = Q'
    R = B
    return(Q,R)
end

(Q3, R3Complete) = QRGivens(A3)

#Ahora podemos encontrar delta como la solución del sistema triangular superior R \delta = Q^T*\ln R
delta = SolBwd(R3Complete[1:4,1:4],((Q3)'*Rlog)[1:4,1] )

lnb3 = delta[1] 
b3 = exp(lnb3)
a3 = delta[2] 
c3 = delta[3] 
d3 = delta[4]

Ecubic = 0
for i = 1:37
   Ecubic = Ecubic + ( R[i] - b3*(W[i]^a3)*exp(c3*(log(W[i])^2))*exp(d3*(log(W[i])^3)))^2
end
println(Ecubic)

130.8219199816802


In [25]:
    println(E)
    println(Equad)
    println(Ecubic)

148.40532603675385
163.1474067643412
130.8219199816802


# Conclusión:

El modelo con término cúbico obtiene la mejor aproximación